In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [48]:
train = pd.read_csv("data/Training.csv")
test = pd.read_csv("data/Testing.csv")
data_train = train
data_test = test

In [49]:
train[:5]

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,Unnamed: 133
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN


In [50]:
data_train = data_train.drop(["Unnamed: 133"],axis=1)

In [51]:
data_train.prognosis.value_counts()

Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
Allergy      

In [52]:
Y = data_train[["prognosis"]]
X = data_train.drop(["prognosis"], axis=1)
Y_pred = data_test.drop(["prognosis"], axis=1)

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [54]:
classifier = DecisionTreeClassifier(random_state=42)
disease_model = classifier.fit(X_train, y_train)

In [56]:
test_pred = disease_model.predict(X_test)
print("testing accuracy is:",accuracy_score(y_test,test_pred))

testing accuracy is: 1.0


In [37]:
#Creating a random row of values to perform model inference on

import random

inferenceRow = X_test.iloc[1:2, ]
inferenceVal = y_test[1:2]
for i in range(132):
  inferenceRow.iloc[0:1, i] = random.randint(0, 1)
# inferenceRow
randomInference = inferenceRow
randomInference

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
4916,1,0,0,1,1,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,1


In [57]:
inferencePred = disease_model.predict(randomInference)
inferencePred

array(['Hyperthyroidism'], dtype=object)

In [64]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [76]:
# Define input for model
inp = "I have a runny nose, cough, and sore throat"
lemmatized_text = " ".join([token.lemma_ for token in nlp(inp)])

In [77]:
# Use symptom extraction with keyBERT to get the symptoms from the text input
from keybert import KeyBERT
model = KeyBERT('distilbert-base-nli-mean-tokens')

keywords = model.extract_keywords(lemmatized_text, keyphrase_ngram_range=(1, 2))
print(keywords)

[('cough sore', 0.8432), ('sore throat', 0.8108), ('nose cough', 0.7859), ('runny nose', 0.7655), ('cough', 0.6732)]


In [41]:
words = [word for word, _ in keywords]
print(words)

['cough', 'sore', 'runny', 'throat', 'nose']


In [26]:
# Make a symptoms dataframe that is in the format for the model
zeros_df = pd.DataFrame({col: [0] for col in X_train.columns})
zeros_df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
symptoms_list = X.columns.tolist()

In [ ]:
# Make symptom synonyms list to check for synonyms/similar wording
symptom_keywords = {
    'itching': ['itch', 'pruritus', 'prickling', 'itchiness', 'tickling',
                'scratching', 'scratch', 'tingling', 'tingle', 'tickle',
                'prickle', 'crawling'],
    'skin_rash': ['rash', 'dermatitis', 'eczema', 'redness', 'bumps',
                  'lumps', 'lesions'],
    '
}

In [46]:
# Change the related symptoms to 1 if one of the keywords extracted are in a symptom name
for word in words:
  found = False
  for label in symptoms_list:
    if ((word == "skin") or (word == "pain") or (word == "nails")):
      pass
    elif word in label:
      found = True
      print(f"{word} was found in the list of labels as {label}")
      zeros_df[label] = 1
    else:
      zeros_df[label] = 0
  if not found:
    print(f"{word} was not found in the list of labels")

cough was found in the list of labels as cough
sore was found in the list of labels as red_sore_around_nose
runny was found in the list of labels as runny_nose
throat was found in the list of labels as patches_in_throat
throat was found in the list of labels as throat_irritation
nose was found in the list of labels as runny_nose
nose was found in the list of labels as red_sore_around_nose


In [58]:
inferencePred = disease_model.predict(zeros_df)
inferencePred

array(['Arthritis'], dtype=object)

In [80]:
from keybert import KeyBERT
import spacy

nlp = spacy.load("en_core_web_sm")
model = KeyBERT('distilbert-base-nli-mean-tokens')

def extract_keywords(text):
    keywords = []
    for token in nlp(text):
        lemma = token.lemma_
        if not lemma.isalpha(): # skip non-alphabetic characters
            continue
        keywords += [word for word, _ in model.extract_keywords(lemma, keyphrase_ngram_range=(1,2))]
    return keywords

text = "I have a runny nose, light cough, and sore throat"
keywords = extract_keywords(text)
print(keywords)

['runny', 'nose', 'light', 'cough', 'sore', 'throat']


In [72]:
!python -m spacy download en_vectors_web_lg


[x] No compatible package found for 'en_vectors_web_lg' (spaCy v3.4.4)

